   <center><h1>This is a dev and test environment for EMD1.0</h1></center>

In [31]:
!pip install py4j
import py4j.java_gateway as jg
from py4j.java_gateway import JavaGateway
import subprocess
import threading
import os
'''Responsible for communicating with the NetLogo controller server, a Java executable'''
class NetLogo_Controller_Connector:
    __gw = JavaGateway()# New gateway connection 
    ##server is in netlogo app folder
    __server_name = "C:/Program Files/NetLogo 6.0.2/app/NetLogoControllerServer.jar"
    
    '''Internal method to start JavaGateway server. Will be called by starServer on seperate thread'''
    def __runServer(self): 
        env = dict(os.environ)
        env['JAVA_OPTS'] = ''
        print(subprocess.call(['java', '-jar', self.__server_name]))
    
    '''Starts JavaGateway server'''
    def startServer(self):
        #Fire up the NetLogo Controller server through python
        thread = threading.Thread(target=self.__runServer, args=())
        thread.start()
    
    '''Start up the model and run. Returns a single metric'''
    def run_model(self,model_string, steps, metric):
        bridge = self.__gw.entry_point
        bridge.refresh()
        print(bridge.openModel(model_string))
        setParamsRandom(self.__gw,bridge, model_string)
        bridge.command("setup")
        bridge.command("repeat " + str(steps) + " [go]")
        result = bridge.report(str(metric))
        bridge.closeModel()
        return result
    
    '''Send shutdown signal to the JavaGateway server. No further communication is possible unless server is restarted'''
    def shutdownServer(self):
        gw.shutdown(True)
    
    '''Sets the parameters randomly through the JavaGateway'''
    def setParamsRandom(self, gateway, bridge, model_string):
        paramSpecs = bridge.getParamList(model_string).getParamSpecs()

        for paramSpec in paramSpecs:
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.DoubleDiscreteSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.DoubleContinuousSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.CategoricalSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
                if type(paramValue) != bool:#isinstance(data[i][k], bool)
                    paramValue = '"{}"'.format(paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast()))
            if jg.is_instance_of(gateway,paramSpec,"bsearch.space.ConstantSpec"):
                paramValue = paramSpec.generateRandomValue(gateway.jvm.org.nlogo.api.MersenneTwisterFast())
            print("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
            bridge.command("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        bridge.command("set " + str(paramSpec.getParameterName()) + " " + str(paramValue))
        bridge.command("set historic-view? true" )
    
    '''Returns the Parameter Specification of the NetLogo Model'''
    def getParamSpecs(self):
        return bridge.getParamList(model_string).getParamSpecs()

'''Responsible for Writing Customized NetLogo models for EMD'''
class NetLogo_Writer:
    __model_string = ""
    EMD_line = -1000
    
    '''locate the NetLogo file, read it in, and identify the line to be experimented with as EMD_line.'''
    def __init__(self, model_string):
        __model_string = model_string
        #find EMD entry point
        with  open(model_string, 'r') as file_reader:#should probably catch an exception here

            for i, line in enumerate(file_reader):
                if ";;insert evolutionary code here" in line:
                    print(i)
                    print(line)
                    EMD_line = i + 1
                
    '''if EMD annotation exists, then inject the new rule string'''
    def injectNewRule (self, new_rule):
        # with is like your try .. finally block in this case
        with open(model_string, 'r') as file:  #should probably catch an exception here
            # read a list of lines into data
            data = file.readlines()
            file.close()

        if EMD_line >= 0:
            print( "Your line: " + data[EMD_line])
            data[EMD_line] = new_rule
            print(data)
            # and write everything back
            with open(model_string + 'modelout.nlogo', 'w') as file:
                file.writelines( data )
                file.flush()
                file.close()

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
'''The ABM with all its flexibility as EMD sees it'''
'''EMD can use this class to control, set parameters, query results from'''
'''and most importantly, change rules within the NetLogo model specified'''
class ABM_NetLogo:
    path_to_model = ""
    model_as_text = ""
    parameters = []
    __nlogo_writer = None 
    def __init__(self, path_to_model):
        self.path_to_model = path_to_model
        __nlogo_writer = NetLogo_Writer(path_to_model)
        #Read in the model to memory (instead of repeated disk reads)
        print()
    def inject_new_rule(self, new_rule):
        #locate tag comment
        __nlogo_writer.injectNewRule(new_rule)
        #Replace line under tag with new_rule
        
        #Write model 
        
        #Flush, close
        print()
    def set_parameters(self, parameter_values):
        print()
    def get_parameter_names(self):
        #Return parameter names
        print()
    def run_abm(self):
        #Run the ABM
        print()


class EMD_Individual:
    def __init__(self):
        print()
    

In [3]:
modelpath = "./models/ArtificialAnasazi/Artificial Anasazi Ver 6.nlogo"
ABM_NetLogo(modelpath)

259
    ;;insert evolutionary code here

()


<__main__.ABM_NetLogo instance at 0x0000000005F67188>

In [32]:
n = NetLogo_Controller_Connector()
n.startServer()

Exception in thread Thread-17:
Traceback (most recent call last):
  File "C:\Users\ch328575\AppData\Local\Continuum\Anaconda2\lib\threading.py", line 801, in __bootstrap_inner
    self.run()
  File "C:\Users\ch328575\AppData\Local\Continuum\Anaconda2\lib\threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "<ipython-input-31-f453659aaf3f>", line 17, in __runServer
    print(subprocess.call(['java', '-jar', serlf.__server_name]))
NameError: global name 'serlf' is not defined

